 # <center> Lesson 3. Logistic regression</center> 

In [62]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
X = np.array([ [   1,    1,  500,    1],
               [   1,    1,  700,    1],
               [   1,    2,  750,    2],
               [   1,    5,  600,    1],
               [   1,    3, 1450,    2],
               [   1,    0,  800,    1],
               [   1,    5, 1500,    3],
               [   1,   10, 2000,    3],
               [   1,    1,  450,    1],
               [   1,    2, 1000,    2]], dtype=np.float64)

y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype=np.float64)

## Task 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

Добавим в аргумент логарифма малое значение, чтобы в случае, когда $y_p \in \{ 0,1 \}$, логарифмы хорошо считались

In [2]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-15) + (1.0 - y) * np.log(1.0 - y_pred + 1e-15))
    return err

## Task 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [95]:
def eval_model(X, y, iterations):
    np.random.seed(42)
    W = np.random.randn(X.shape[1])
    n = X.shape[0]
    errors =[]
    
    for eta in [1e-4,1e-3,1e-2,1e-4,2e-3,2e-2]:
        error_eta = []
        for i in range(iterations):
            z = np.dot(X, W)
            y_pred = 1 / (1 + np.exp(-z))
            err = calc_logloss(y, y_pred)
                
            dQ = 1/n * X.T @ (y_pred - y)
            W -= eta * dQ
        error_eta.append(err)
        errors.append(error_eta)
    return errors, W

In [96]:
er, W = eval_model(X, y, iterations = 700)
er

[[1.010139164862049],
 [0.5266929942065267],
 [0.48210013277905056],
 [0.4818824954269882],
 [0.4782889778897646],
 [0.45201114382587565]]

Берем $\eta = 0.02$

## Task 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [101]:
def calc_pred_proba(W, X):
 
    y_predicted = np.zeros(X.shape[0])
    y_pred_proba = np.squeeze(1 / (1 + np.exp(-np.dot(X, W))))
    return y_pred_proba     

In [102]:
y_pred_proba = calc_pred_proba(W, X) 
y_pred_proba

array([0.36397081, 0.46166112, 0.71900805, 0.0239606 , 0.82017739,
       0.7081455 , 0.85482526, 0.19706667, 0.34089705, 0.80925685])

## Task 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [103]:
def calc_pred(W, X):
    
    y_predicted = np.zeros(X.shape[0])
    y_pred_proba = np.squeeze(1 / (1 + np.exp(-np.dot(X, W))))

    # За порог отнесения к тому или иному классу примем вероятность 0.5
    for i in range(y_pred_proba.shape[0]):
        if (y_pred_proba[i] > 0.5): 
            y_predicted[i] = 1
        elif (y_pred_proba[i] <= 0.5):
            y_predicted[i] = 0

    return y_predicted

In [104]:
y_predicted = calc_pred(W, X) 
y_predicted

array([0., 0., 1., 0., 1., 1., 1., 0., 0., 1.])

In [152]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

## Task 5. *Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

In [114]:
def accuracy(y_pred,y):
    true_pred = 0
    for i in range(len(y)):
        if y_pred[i] == y[i]:
            true_pred += 1
    return true_pred/len(y)

In [182]:
def precision(y_pred,y):
    res = {'true_pos': 0, 'false_pos': 0, "true_neg": 0, 'false_neg': 0}

    for i in range(y.shape[0]):
        if (y_pred[i] == y[i]) and y[i] == 1:
            res['true_pos'] += 1
        elif (y_pred[i] == y[i]) and y[i] == 0:
            res['true_neg'] += 0
        elif (y_pred[i] != y[i]) and y[i] == 0:
            res['false_pos'] += 0
        else:
            res['false_neg'] += 1
    #return res, res['true_pos']/(res['true_pos'] + res['false_pos'])      
    return res

In [173]:
def recall(y_pred,y):
    res = {'true_pos': 0, 'false_pos': 0, "true_neg": 0, 'false_neg': 0}

    for i in range(len(y)):
        if (y_pred[i] == y[i]) and y[i] == 1:
            res['true_pos'] += 1
        elif (y_pred[i] == y[i]) and y[i] == 0:
            res['true_neg'] += 0
        elif (y_pred[i] != y[i]) and y[i] == 0:
            res['false_pos'] += 0
        elif (y_pred[i] != y[i]) and y[i] == 1:
            res['false_neg'] += 1
    return res['true_pos']/(res['true_pos'] + res['true_neg'])

### Confusion matrix

In [202]:
def conf_matrix(y_pred,y):
    res = {'true_pos': 0, 'false_pos': 0, "true_neg": 0, 'false_neg': 0}

    for i in range(len(y)):
        if (y_pred[i] == y[i]) and y[i] == 1:
            res['true_pos'] += 1
        elif (y_pred[i] == y[i]) and y[i] == 0:
            res['true_neg'] += 0
        elif (y_pred[i] != y[i]) and y[i] == 0:
            res['false_pos'] += 0
        elif (y_pred[i] != y[i]) and y[i] == 1:
            res['false_neg'] += 1
    print (f"|          |y = 1|y = 0|\n y_pred = 1|{res['true_pos']}    |{res['false_pos']}    |\n y_pred = 0|{res['false_neg']}    |{res['true_neg']}    |")


### F1-score

In [116]:
def f1_score(precision,recall):
    return (2 * precision * recall) / (precision + recall)

## Task 6. Могла ли модель переобучиться? Почему?

Да,  логистическая регрессия всё та же линейная модель, а значит
1. для неё важно качество входящих данных (предварительно следует данные стандартизировать/нормализовать);
2. может переобучаться, если не ограничивать норму вектора весов (регуляризация).